In [8]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adagrad, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from keras import regularizers
from keras.models import Sequential
from keras.layers import Embedding
from keras.optimizers import SGD, Adagrad, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
stop_words = set(stopwords.words("english"))

def preprocess(text):
    # Remove @username
    text = re.sub(r'@\w+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-z0-9\s]', '', text)
    # Tokenize the words
    words = word_tokenize(text)
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    # Join the words
    text = ' '.join(lemmatized_words)
    return text




data = pd.read_csv("Tweets.csv")
df = data[["text","airline_sentiment"]]
df = df[df.airline_sentiment != "neutral"]
df['text'] = df['text'].apply(preprocess)



max_features = 4000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['text'].values)

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)
Y = np.array(df['airline_sentiment'] == "positive", dtype=int)


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)



In [10]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers

In [11]:
def build_classifier(optimizer):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(max_features, 128, input_length=X_train.shape[1]))
    model.add(tf.keras.layers.SpatialDropout1D(0.5))
    model.add(tf.keras.layers.LSTM(196, dropout = 0.3, recurrent_dropout = 0.3 ))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(100, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax,kernel_regularizer=regularizers.L1(0.01)))

    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [12]:
model = KerasClassifier(build_fn=build_classifier)

parameters = {'batch_size': [16,32],
              'epochs': [8,10],
              'optimizer': ['adam','rmsprop']}


C:\Users\prana\AppData\Local\Temp\ipykernel_6252\3254356261.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_classifier)


In [13]:


grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)

In [14]:


grid_search = grid_search.fit(X_train, Y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_



Epoch 1/8
404/404 [==============================] - 19s 43ms/step - loss: 0.4539 - accuracy: 0.8497
Epoch 2/8
404/404 [==============================] - 17s 43ms/step - loss: 0.2398 - accuracy: 0.9242
Epoch 3/8
404/404 [==============================] - 17s 43ms/step - loss: 0.1864 - accuracy: 0.9434
Epoch 4/8
404/404 [==============================] - 17s 43ms/step - loss: 0.1490 - accuracy: 0.9530
Epoch 5/8
404/404 [==============================] - 17s 43ms/step - loss: 0.1267 - accuracy: 0.9630
Epoch 6/8
404/404 [==============================] - 17s 42ms/step - loss: 0.1055 - accuracy: 0.9681
Epoch 7/8
404/404 [==============================] - 17s 43ms/step - loss: 0.0988 - accuracy: 0.9698
Epoch 8/8
51/51 [==============================] - 1s 9ms/step
Epoch 1/8
404/404 [==============================] - 18s 43ms/step - loss: 0.4545 - accuracy: 0.8538
Epoch 2/8
404/404 [==============================] - 17s 43ms/step - loss: 0.2400 - accuracy: 0.9197
Epoch 3/8
404/404 [=========

404/404 [==============================] - 17s 43ms/step - loss: 0.2182 - accuracy: 0.9246
Epoch 6/8
404/404 [==============================] - 17s 43ms/step - loss: 0.2148 - accuracy: 0.9228
Epoch 7/8
404/404 [==============================] - 17s 43ms/step - loss: 0.1994 - accuracy: 0.9305
Epoch 8/8
51/51 [==============================] - 1s 9ms/step
Epoch 1/10
404/404 [==============================] - 18s 43ms/step - loss: 0.4591 - accuracy: 0.8517
Epoch 2/10
404/404 [==============================] - 17s 43ms/step - loss: 0.2412 - accuracy: 0.9236
Epoch 3/10
404/404 [==============================] - 17s 43ms/step - loss: 0.1858 - accuracy: 0.9432
Epoch 4/10
404/404 [==============================] - 17s 43ms/step - loss: 0.1501 - accuracy: 0.9522
Epoch 5/10
404/404 [==============================] - 17s 43ms/step - loss: 0.1279 - accuracy: 0.9602
Epoch 6/10
404/404 [==============================] - 17s 43ms/step - loss: 0.1022 - accuracy: 0.9681
Epoch 7/10
404/404 [============

404/404 [==============================] - 17s 43ms/step - loss: 0.2669 - accuracy: 0.9070
Epoch 4/10
404/404 [==============================] - 17s 43ms/step - loss: 0.2465 - accuracy: 0.9126
Epoch 5/10
404/404 [==============================] - 17s 43ms/step - loss: 0.2321 - accuracy: 0.9192
Epoch 6/10
404/404 [==============================] - 17s 43ms/step - loss: 0.2221 - accuracy: 0.9259
Epoch 7/10
404/404 [==============================] - 17s 43ms/step - loss: 0.2106 - accuracy: 0.9246
Epoch 8/10
404/404 [==============================] - 17s 43ms/step - loss: 0.2056 - accuracy: 0.9290
Epoch 9/10
404/404 [==============================] - 17s 43ms/step - loss: 0.1977 - accuracy: 0.9282
Epoch 10/10
51/51 [==============================] - 1s 9ms/step
Epoch 1/10
404/404 [==============================] - 18s 43ms/step - loss: 0.4734 - accuracy: 0.8428
Epoch 2/10
404/404 [==============================] - 17s 43ms/step - loss: 0.3071 - accuracy: 0.8954
Epoch 3/10
404/404 [========

202/202 [==============================] - 20s 91ms/step - loss: 0.5193 - accuracy: 0.8389
Epoch 2/8
202/202 [==============================] - 18s 91ms/step - loss: 0.3089 - accuracy: 0.9016
Epoch 3/8
202/202 [==============================] - 18s 91ms/step - loss: 0.2618 - accuracy: 0.9141
Epoch 4/8
202/202 [==============================] - 18s 91ms/step - loss: 0.2339 - accuracy: 0.9226
Epoch 5/8
202/202 [==============================] - 18s 91ms/step - loss: 0.2120 - accuracy: 0.9277
Epoch 6/8
202/202 [==============================] - 18s 91ms/step - loss: 0.2067 - accuracy: 0.9291
Epoch 7/8
202/202 [==============================] - 18s 91ms/step - loss: 0.1965 - accuracy: 0.9341
Epoch 8/8
51/51 [==============================] - 1s 21ms/step
Epoch 1/8
202/202 [==============================] - 21s 92ms/step - loss: 0.5231 - accuracy: 0.8366
Epoch 2/8
202/202 [==============================] - 19s 92ms/step - loss: 0.3079 - accuracy: 0.9006
Epoch 3/8
202/202 [==================

202/202 [==============================] - 18s 91ms/step - loss: 0.1273 - accuracy: 0.9635
Epoch 6/10
202/202 [==============================] - 18s 91ms/step - loss: 0.1133 - accuracy: 0.9678
Epoch 7/10
202/202 [==============================] - 18s 91ms/step - loss: 0.0963 - accuracy: 0.9745
Epoch 8/10
202/202 [==============================] - 19s 92ms/step - loss: 0.0805 - accuracy: 0.9800
Epoch 9/10
202/202 [==============================] - 19s 92ms/step - loss: 0.0785 - accuracy: 0.9785
Epoch 10/10
51/51 [==============================] - 1s 22ms/step
Epoch 1/10
202/202 [==============================] - 20s 91ms/step - loss: 0.5202 - accuracy: 0.8400
Epoch 2/10
202/202 [==============================] - 18s 90ms/step - loss: 0.3158 - accuracy: 0.8971
Epoch 3/10
202/202 [==============================] - 18s 90ms/step - loss: 0.2583 - accuracy: 0.9175
Epoch 4/10
202/202 [==============================] - 18s 90ms/step - loss: 0.2334 - accuracy: 0.9203
Epoch 5/10
202/202 [=======

In [15]:
print("Best parameters: ", best_parameters)
print("Best accuracy: ", best_accuracy)


Best parameters:  {'batch_size': 32, 'epochs': 10, 'optimizer': 'rmsprop'}
Best accuracy:  0.9083916255402631


# Ensemble Technique to Handle Imbalance dataset problem:

In [17]:
# function to average the predictions of individual models
def ensemble_predict(models, X):
    predictions = [model.predict(X) for model in models]
    avg_prediction = np.average(predictions, axis=0)
    return avg_prediction

# create a list to store the individual models
models = []

# number of models to train
num_models = 5

for i in range(num_models):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(max_features, 128, input_length=X_train.shape[1]),
        tf.keras.layers.SpatialDropout1D(0.5),
        tf.keras.layers.LSTM(196, dropout=0.3, recurrent_dropout=0.3),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(100, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(2, activation=tf.nn.softmax,kernel_regularizer=regularizers.L1(0.01))
    ])
    model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.3, verbose=2)
    models.append(model)


Epoch 1/10
177/177 - 20s - loss: 0.5464 - accuracy: 0.8352 - val_loss: 0.4348 - val_accuracy: 0.8700 - 20s/epoch - 111ms/step
Epoch 2/10
177/177 - 17s - loss: 0.3190 - accuracy: 0.8979 - val_loss: 0.3437 - val_accuracy: 0.8923 - 17s/epoch - 98ms/step
Epoch 3/10
177/177 - 17s - loss: 0.2604 - accuracy: 0.9163 - val_loss: 0.3307 - val_accuracy: 0.8923 - 17s/epoch - 98ms/step
Epoch 4/10
177/177 - 17s - loss: 0.2238 - accuracy: 0.9259 - val_loss: 0.2985 - val_accuracy: 0.8944 - 17s/epoch - 98ms/step
Epoch 5/10
177/177 - 17s - loss: 0.2089 - accuracy: 0.9310 - val_loss: 0.3067 - val_accuracy: 0.8973 - 17s/epoch - 98ms/step
Epoch 6/10
177/177 - 17s - loss: 0.1924 - accuracy: 0.9358 - val_loss: 0.2772 - val_accuracy: 0.8965 - 17s/epoch - 98ms/step
Epoch 7/10
177/177 - 17s - loss: 0.1866 - accuracy: 0.9369 - val_loss: 0.2843 - val_accuracy: 0.8940 - 17s/epoch - 98ms/step
Epoch 8/10
177/177 - 17s - loss: 0.1729 - accuracy: 0.9432 - val_loss: 0.2906 - val_accuracy: 0.8948 - 17s/epoch - 98ms/step

In [18]:
batch_size = 32
ensemble_inputs = models[0].input
input_shape = models[0].input_shape[1:]
X = np.zeros((batch_size,) + input_shape)
ensemble_outputs = [model(ensemble_inputs) for model in models]
average_outputs = tf.reduce_mean(ensemble_outputs, axis=0)
average_outputs = tf.keras.layers.Lambda(lambda x: x)(average_outputs)
ensemble_model = tf.keras.models.Model(inputs=ensemble_inputs, outputs=average_outputs)

In [19]:
ensemble_model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [22]:
ensemble_model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.3, verbose=2)

Epoch 1/10
177/177 - 82s - loss: 0.1574 - accuracy: 0.9540 - val_loss: 0.3195 - val_accuracy: 0.8998 - 82s/epoch - 464ms/step
Epoch 2/10
177/177 - 82s - loss: 0.1496 - accuracy: 0.9568 - val_loss: 0.3392 - val_accuracy: 0.8940 - 82s/epoch - 465ms/step
Epoch 3/10
177/177 - 82s - loss: 0.1444 - accuracy: 0.9593 - val_loss: 0.3422 - val_accuracy: 0.8981 - 82s/epoch - 463ms/step
Epoch 4/10
177/177 - 82s - loss: 0.1416 - accuracy: 0.9586 - val_loss: 0.3276 - val_accuracy: 0.8936 - 82s/epoch - 464ms/step
Epoch 5/10
177/177 - 82s - loss: 0.1366 - accuracy: 0.9627 - val_loss: 0.3846 - val_accuracy: 0.8965 - 82s/epoch - 464ms/step
Epoch 6/10
177/177 - 82s - loss: 0.1306 - accuracy: 0.9641 - val_loss: 0.4071 - val_accuracy: 0.8919 - 82s/epoch - 463ms/step
Epoch 7/10
177/177 - 82s - loss: 0.1298 - accuracy: 0.9643 - val_loss: 0.3220 - val_accuracy: 0.8903 - 82s/epoch - 464ms/step
Epoch 8/10
177/177 - 82s - loss: 0.1215 - accuracy: 0.9669 - val_loss: 0.3762 - val_accuracy: 0.8915 - 82s/epoch - 464

In [23]:

# evaluate the ensemble model
ensemble_model.evaluate(X_test, Y_test)


109/109 [==============================] - 11s 98ms/step - loss: 0.3343 - accuracy: 0.9108


[0.3343445956707001, 0.9107710123062134]

In [24]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve,auc

In [25]:
Y_pred = ensemble_model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred, axis=1)

precision = precision_score(Y_test, Y_pred_classes)
recall = recall_score(Y_test, Y_pred_classes)
f1 = f1_score(Y_test, Y_pred_classes)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

fpr, tpr, thresholds = roc_curve(Y_test, Y_pred[:, 1])
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)


109/109 [==============================] - 11s 96ms/step
Precision: 0.7769679300291545
Recall: 0.7735849056603774
F1-Score: 0.7752727272727273
AUC: 0.9451228125984286


# Update for Cost-Sensitive Learning

In [26]:
def get_weights(Y):
    class_weights = np.zeros(2)
    for i in range(2):
        class_weights[i] = 1.0 / (sum(Y == i)/len(Y))
    return class_weights

class_weights = get_weights(Y_train)
print("Class Weights: ", class_weights)

ensemble_model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"],
sample_weight_mode="temporary", weighted_metrics=["accuracy"], loss_weights=class_weights[1])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)


ensemble_model.fit(X_train, Y_train, epochs=100, batch_size=64, validation_split=0.4, verbose=2)


Class Weights:  [1.26139913 4.8255675 ]
Epoch 1/100
76/76 - 60s - loss: 0.4763 - accuracy: 0.9688 - weighted_accuracy: 0.9688 - val_loss: 1.5432 - val_accuracy: 0.9115 - val_weighted_accuracy: 0.9115 - 60s/epoch - 791ms/step
Epoch 2/100
76/76 - 48s - loss: 0.4630 - accuracy: 0.9719 - weighted_accuracy: 0.9719 - val_loss: 1.3649 - val_accuracy: 0.9155 - val_weighted_accuracy: 0.9155 - 48s/epoch - 638ms/step
Epoch 3/100
76/76 - 49s - loss: 0.4460 - accuracy: 0.9715 - weighted_accuracy: 0.9715 - val_loss: 1.4294 - val_accuracy: 0.9025 - val_weighted_accuracy: 0.9025 - 49s/epoch - 639ms/step
Epoch 4/100
76/76 - 49s - loss: 0.4203 - accuracy: 0.9738 - weighted_accuracy: 0.9738 - val_loss: 1.4354 - val_accuracy: 0.9134 - val_weighted_accuracy: 0.9134 - 49s/epoch - 639ms/step
Epoch 5/100
76/76 - 49s - loss: 0.3938 - accuracy: 0.9734 - weighted_accuracy: 0.9734 - val_loss: 1.7857 - val_accuracy: 0.9158 - val_weighted_accuracy: 0.9158 - 49s/epoch - 639ms/step
Epoch 6/100
76/76 - 49s - loss: 0.3

Epoch 45/100
76/76 - 49s - loss: 0.1105 - accuracy: 0.9953 - weighted_accuracy: 0.9953 - val_loss: 2.5507 - val_accuracy: 0.8994 - val_weighted_accuracy: 0.8994 - 49s/epoch - 639ms/step
Epoch 46/100
76/76 - 49s - loss: 0.1122 - accuracy: 0.9948 - weighted_accuracy: 0.9948 - val_loss: 2.4550 - val_accuracy: 0.8970 - val_weighted_accuracy: 0.8970 - 49s/epoch - 639ms/step
Epoch 47/100
76/76 - 49s - loss: 0.1101 - accuracy: 0.9950 - weighted_accuracy: 0.9950 - val_loss: 2.6583 - val_accuracy: 0.8982 - val_weighted_accuracy: 0.8982 - 49s/epoch - 639ms/step
Epoch 48/100
76/76 - 49s - loss: 0.1122 - accuracy: 0.9953 - weighted_accuracy: 0.9953 - val_loss: 2.0482 - val_accuracy: 0.8970 - val_weighted_accuracy: 0.8970 - 49s/epoch - 640ms/step
Epoch 49/100
76/76 - 49s - loss: 0.1080 - accuracy: 0.9948 - weighted_accuracy: 0.9948 - val_loss: 2.6462 - val_accuracy: 0.8979 - val_weighted_accuracy: 0.8979 - 49s/epoch - 639ms/step
Epoch 50/100
76/76 - 49s - loss: 0.1066 - accuracy: 0.9955 - weighted_

76/76 - 49s - loss: 0.0627 - accuracy: 0.9973 - weighted_accuracy: 0.9973 - val_loss: 2.6690 - val_accuracy: 0.8973 - val_weighted_accuracy: 0.8973 - 49s/epoch - 639ms/step
Epoch 90/100
76/76 - 49s - loss: 0.0565 - accuracy: 0.9969 - weighted_accuracy: 0.9969 - val_loss: 2.8108 - val_accuracy: 0.8982 - val_weighted_accuracy: 0.8982 - 49s/epoch - 639ms/step
Epoch 91/100
76/76 - 49s - loss: 0.0588 - accuracy: 0.9973 - weighted_accuracy: 0.9973 - val_loss: 2.7045 - val_accuracy: 0.8951 - val_weighted_accuracy: 0.8951 - 49s/epoch - 639ms/step
Epoch 92/100
76/76 - 49s - loss: 0.0663 - accuracy: 0.9977 - weighted_accuracy: 0.9977 - val_loss: 2.6800 - val_accuracy: 0.8960 - val_weighted_accuracy: 0.8960 - 49s/epoch - 640ms/step
Epoch 93/100
76/76 - 49s - loss: 0.0588 - accuracy: 0.9969 - weighted_accuracy: 0.9969 - val_loss: 2.8025 - val_accuracy: 0.8973 - val_weighted_accuracy: 0.8973 - 49s/epoch - 639ms/step
Epoch 94/100
76/76 - 49s - loss: 0.0558 - accuracy: 0.9973 - weighted_accuracy: 0.9

In [27]:
score = ensemble_model.evaluate(X_test, Y_test, verbose=False)
print("loss = ", score[0])
print("accuracy = ", score[1])

loss =  2.70139741897583
accuracy =  0.8977764844894409


In [28]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve,auc

In [29]:
Y_pred = ensemble_model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred, axis=1)

precision = precision_score(Y_test, Y_pred_classes)
recall = recall_score(Y_test, Y_pred_classes)
f1 = f1_score(Y_test, Y_pred_classes)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

fpr, tpr, thresholds = roc_curve(Y_test, Y_pred[:, 1])
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)


109/109 [==============================] - 12s 99ms/step
Precision: 0.7474150664697193
Recall: 0.7343976777939042
F1-Score: 0.7408491947291361
AUC: 0.9331915788636552
